<a href="https://colab.research.google.com/github/p-tech/wbs-dm/blob/main/Create-Data/Create_fake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

def create_database():
    # Connect to SQLite database (creates a new one if it doesn't exist)
    conn = sqlite3.connect('mydb.sqlite')
    cursor = conn.cursor()

    # Enable foreign keys support in SQLite
    cursor.execute("PRAGMA foreign_keys = ON;")

    # Create Owner table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Owner (
        idOwner INTEGER PRIMARY KEY,
        Name TEXT,
        Address TEXT,
        Telephone TEXT
    );
    ''')

    # Create Pets table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Pets (
        idPets INTEGER PRIMARY KEY,
        Name TEXT,
        date_of_birth TEXT,
        breed TEXT,
        Owner_idOwner INTEGER,
        FOREIGN KEY (Owner_idOwner) REFERENCES Owner (idOwner) ON DELETE NO ACTION ON UPDATE NO ACTION
    );
    ''')

    # Commit and close
    conn.commit()
    conn.close()
    print("Database and tables created successfully.")

# Run the function to create the database
create_database()

Database and tables created successfully.


In [3]:
# prompt: using faker create 10 emplyees in different departments, create teh relevant tables to normalise the employee into a department and insert the created data into the tables withthe correct PK and FK
!pip install faker
import sqlite3
from faker import Faker

fake = Faker()

def create_employee_data(num_employees):
    employees = []
    departments = ["Sales", "Marketing", "Engineering", "HR", "Finance"]
    for _ in range(num_employees):
        employees.append({
            'name': fake.name(),
            'department': fake.random_element(elements=departments)
        })
    return employees

def create_tables():
    conn = sqlite3.connect('company.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS departments (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            department_id INTEGER,
            FOREIGN KEY (department_id) REFERENCES departments(id)
        )
    ''')
    conn.commit()
    conn.close()

def insert_data(employees):
    conn = sqlite3.connect('company.db')
    cursor = conn.cursor()

    departments_dict = {}

    for employee in employees:
        if employee['department'] not in departments_dict:
            cursor.execute("INSERT OR IGNORE INTO departments (name) VALUES (?)", (employee['department'],))
            departments_dict[employee['department']] = cursor.lastrowid

        department_id = departments_dict[employee['department']]
        cursor.execute("INSERT INTO employees (name, department_id) VALUES (?, ?)", (employee['name'], department_id))

    conn.commit()
    conn.close()

# Generate employee data
employees = create_employee_data(10)

# Create tables if not exists
create_tables()

# Insert data into tables
insert_data(employees)

print("Data inserted successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.8 MB/s eta 0:00:00
Data inserted successfully!


In [4]:
# prompt: display teh contetns fo the tables

import sqlite3

def display_table_contents(db_name, table_name):
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        if rows:
            print(f"Contents of table '{table_name}':")
            for row in rows:
                print(row)
        else:
            print(f"Table '{table_name}' is empty.")

        conn.close()

    except sqlite3.Error as e:
        print(f"Error: {e}")

# Display contents of the 'employees' table in 'company.db'
display_table_contents('company.db', 'employees')

# Display contents of the 'departments' table in 'company.db'
display_table_contents('company.db', 'departments')


Contents of table 'employees':
(1, 'Mr. Christopher Garcia', 1)
(2, 'Kimberly Allison', 2)
(3, 'Kelly Campbell', 3)
(4, 'Melinda Craig', 2)
(5, 'Donald Davis', 3)
(6, 'Monique Zhang', 4)
(7, 'Stacey Mills', 1)
(8, 'Daniel Pace', 1)
(9, 'Patricia Howard', 3)
(10, 'Zachary Moody', 5)
Contents of table 'departments':
(1, 'Marketing')
(2, 'Sales')
(3, 'HR')
(4, 'Engineering')
(5, 'Finance')
